# Classification

## Introduction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Processing

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import tree

In [ ]:
df_train_raw = pd.read_csv('../data/classification/train.csv', index_col=0);
df_train_raw

,date,hour,bc_price,bc_demand,ab_price,ab_demand,transfer,bc_price_evo
id,,,,,,,,
0,0.452060,1.000000,0.074096,0.578846,0.005029,0.494821,0.489912,UP
1,0.455555,0.574468,0.033025,0.349003,0.001554,0.264889,0.829386,DOWN
2,0.027521,0.617021,0.098325,0.533918,0.003467,0.422915,0.414912,UP
3,0.455732,0.936170,0.041822,0.588515,0.002860,0.448731,0.525000,UP
4,0.000044,0.255319,0.051489,0.302440,0.003467,0.422915,0.414912,UP
...,...,...,...,...,...,...,...,...
28850,0.026503,1.000000,0.082232,0.427551,0.003467,0.422915,0.414912,UP
28851,0.451927,0.574468,0.033626,0.564564,0.002198,0.624806,0.553947,DOWN
28852,0.907482,0.893617,0.055872,0.329664,0.003695,0.316416,0.602193,DOWN


In [3]:
df_train = df_train_raw.drop(columns=['bc_price_evo'])
df_train_labels = df_train_raw.loc[:, 'bc_price_evo'].copy()
display(df_train)
df_train_labels

,date,hour,bc_price,bc_demand,ab_price,ab_demand,transfer
id,,,,,,,
0,0.452060,1.000000,0.074096,0.578846,0.005029,0.494821,0.489912
1,0.455555,0.574468,0.033025,0.349003,0.001554,0.264889,0.829386
2,0.027521,0.617021,0.098325,0.533918,0.003467,0.422915,0.414912
3,0.455732,0.936170,0.041822,0.588515,0.002860,0.448731,0.525000
4,0.000044,0.255319,0.051489,0.302440,0.003467,0.422915,0.414912
...,...,...,...,...,...,...,...
28850,0.026503,1.000000,0.082232,0.427551,0.003467,0.422915,0.414912
28851,0.451927,0.574468,0.033626,0.564564,0.002198,0.624806,0.553947
28852,0.907482,0.893617,0.055872,0.329664,0.003695,0.316416,0.602193


id
0          UP
1        DOWN
2          UP
3          UP
4          UP
         ... 
28850      UP
28851    DOWN
28852    DOWN
28853    DOWN
28854    DOWN
Name: bc_price_evo, Length: 28855, dtype: object

In [4]:
df_test = pd.read_csv('../data/classification/test.csv', index_col=0)

In [5]:
def save_submission( df_test_labels, name_model  ):
    test = df_test.copy()
    test['bc_price_evo'] = df_test_labels
    test.to_csv(f'../data/classification/submission/{name_model}.csv', columns=['bc_price_evo'])

## Logistic Regression (Son)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_train, df_train_labels, test_size=0.20, random_state=23)

# clf = LogisticRegression(max_iter=10000, random_state=0) # 74.01%
# clf = LogisticRegression(max_iter=10000, random_state=0, solver='liblinear', dual=True) # 73.94%

# clf = LogisticRegression(max_iter=10000, random_state=0, C=0.01) # 62.97%
# clf = LogisticRegression(max_iter=10000, random_state=0, C=0.1) # 66.77%
# clf = LogisticRegression(max_iter=10000, random_state=0, C=10) # 75.05%
# clf = LogisticRegression(max_iter=10000, random_state=0, C=100) # 74.96%
# clf = LogisticRegression(max_iter=10000, random_state=0, C=15) # 75.12%

# clf = LogisticRegression(max_iter=10000, random_state=0, C=15, penalty='l1', solver='liblinear') # 75%
# clf = LogisticRegression(max_iter=10000, random_state=0, C=15, penalty='elasticnet', solver='saga', l1_ratio=0.5) # 75.08%
# clf = LogisticRegression(max_iter=10000, random_state=0, C=15, class_weight='balanced') # 75.13%
# clf = LogisticRegression(max_iter=10000, random_state=0, C=15, class_weight='balanced', fit_intercept=False) # 71.65%

clf = LogisticRegression(max_iter=10000, random_state=0, C=15, class_weight='balanced') # 75.13%
clf.fit(X_train, y_train)

acc = accuracy_score(y_test, clf.predict(X_test)) * 100
print(f"Logistic Regression model accuracy: {acc:.2f}%")

Logistic Regression model accuracy: 75.13%


In [8]:
clf = LogisticRegression(max_iter=10000, random_state=0, C=15, class_weight='balanced')
clf.fit(df_train, df_train_labels)

test_labels = clf.predict(df_test)

In [9]:
#concat test_labels to df_test index
save_submission(test_labels, 'LogisticRegression')

## Decision Tree (Tu)

In [10]:
dtree = tree.DecisionTreeClassifier()
dtree.fit(df_train, df_train_labels)

DecisionTreeClassifier()

## Random Forest (Tho)


## Support Vector Machine (SVM) (Son)


## Naive Bayes (Tu)


## K-Nearest Neighbors (KNN) (Tho)